<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: Maria Paz Raveau</em><br>
    <em>Asignatura: Procesamiento de Lenguaje Natural</em><br>

</div>

# **Tarea 1**

*Fecha de Entrega: Domingo 20, Julio 2025.*

**Nombre Estudiante**: Victor Saldivia Vera

### **01. Intrucciones y Enunciado**

Suponga que ud. es contactado por alguna agencia gubernamental que quiere saber qué dijeron los participantes de los cabildos de 2016. Ud. debe entonces, basándose en los textos de fundamento, responder a este requerimiento.  Use para ello la base de datos que vimos en clases, de los cabildos provinciales. 

- Escoja alguna de los conceptos que vienen en la base de datos (evite las que no tengan muchas entradas). Hemos visto cómo generar WordClouds con tokens y con bigramas. *¿Es posible valerse de las etiquetas gramaticales para mejorar el resultado?* 
- Pruebe haciendo un WordCloud con tokens/sustantivos,  y otro con  bigramas/sustantivo-adjetivo. Recuerde pre-procesar correctamente.
- Con estos mismo bigramas/sustantivo-adjetivo, haga una red de bigramas.
- Ayúdese con frases de ejemplo, y de los WordClouds, para escribir una descripción del concepto.


### **02. Descripción de la Data**


In [1]:
import pandas as pd
df = pd.read_excel('data/resultadocabildoprovincial.xlsx')
print(f'Dimensión: {df.shape}'), 
print(f'Nombre de Columnas: {df.columns}')

Dimensión: (30414, 12)
Nombre de Columnas: Index(['region_nombre', 'provincia_nombre', 'comuna_nombre', 'cabildo_id',
       'cabildo_nombre', 'cabildo_direccion', 'num_mesa', 'pregunta_glosa',
       'concepto_nombre', 'concepto_otro', 'tipo-eleccion', 'fundamento'],
      dtype='object')


### **03. Librerías a Utilizar**


In [16]:
import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter

# Librerías de Procesamiento de Texto
import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import stanza

# Visualización
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import networkx as nx

# La librería de warnings se utiliza para manejar las advertencias
import warnings
warnings.filterwarnings('ignore')

# Paquete muestra Barra de Progreso
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Descargas (Nota: Ejecutar una sola vez en entorno local)
# nltk.download('punkt')
# nltk.download('stopwords')
# stanza.download('es')

#### Configuración de Stanza para el PLN

In [17]:
nlp = stanza.Pipeline('es', processors='tokenize,pos,lemma', use_gpu=False)  # Cambiar a True si hay GPU


2025-07-20 14:43:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-20 14:43:45 INFO: Downloaded file to C:\Users\Victo\stanza_resources\resources.json
2025-07-20 14:43:45 WARNING: Language es package default expects mwt, which has been added
2025-07-20 14:43:45 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2025-07-20 14:43:45 INFO: Using device: cpu
2025-07-20 14:43:45 INFO: Loading: tokenize
2025-07-20 14:43:45 INFO: Loading: mwt
2025-07-20 14:43:45 INFO: Loading: pos
2025-07-20 14:43:48 INFO: Loading: lemma
2025-07-20 14:43:49 INFO: Done loading processors!


### **04. Carga y Exploración Inicial**


In [18]:
df.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento,14,2
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...,66,11
7,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,"¿Cuáles son los derechos, que la Constitución ...",A la vida,NaN,A,Sin furdamento,14,2
26,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son las instituciones del Estado que d...,Gobierno regional,NaN,A,Sin fundamentos,15,2


#### Conteo de nulos en columnas de texto


In [19]:
texto_col = "fundamento"
print(df[texto_col].isna().sum())

0


#### Estadísticas Descriptivas sobre el texto


In [20]:
df['len_chars'] = df[texto_col].str.len() # columna de longitud de caracteres
df['len_tokens_raw'] = df[texto_col].str.split().str.len() # columna de tokens sin procesar
df[['len_chars','len_tokens_raw']].describe()

,len_chars,len_tokens_raw
count,19326.000000,19326.000000
mean,100.842337,15.763014
std,68.723444,10.954642
min,1.000000,1.000000
25%,50.000000,8.000000
50%,84.000000,13.000000
75%,134.000000,21.000000
max,300.000000,61.000000


#### Eliminación de nulos y duplicados


In [21]:
before = len(df)
df = df.dropna(subset=[texto_col])
df = df.drop_duplicates(subset=[texto_col])
print(f"Filas eliminadas: {before - len(df)}")

Filas eliminadas: 0


### **05. Selección de Concepto**


In [22]:
# Distribución de frecuencias para elegir uno con suficientes observaciones
concepto_col = "concepto_nombre" 
concept_counts = df[concepto_col].value_counts()
concept_counts.head(20)

concepto_nombre
Otro                                                      3976
Igualdad                                                   519
Democracia                                                 507
Descentralización                                          503
Justicia                                                   492
Respeto/conservación de la naturaleza o medio ambiente     484
A la vida                                                  440
A la salud                                                 437
A la educación                                             428
Respeto                                                    385
Fuerzas Armadas                                            381
Bien Común/comunidad                                       381
Congreso o parlamento (estructura y funciones)             373
A la vivienda digna                                        365
Plebiscitos, referendos y consultas                        361
Respeto por la constitución            

#### Selección de Concepto

In [23]:
concepto_elegido = "Democracia" 

#### Creación del Subconjunto filtrado
Se elige el concepto 'Democracia' el cual tiene una frecuencia de (507 ocurrencias) para garantizar masa textual suficiente y diversidad léxica. Se descartaron conceptos con baja frecuencia para evitar resultados inestables en nubes de palabras y bigramas.”



In [24]:
subset = df[df[concepto_col] == concepto_elegido].copy()
print("Filas del concepto:", len(subset))


Filas del concepto: 507


In [25]:
# Subconjunto de datos para el concepto elegido
subset.head(10)

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2
142,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",5,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Que el ciudadano pueda participar en la toma d...,141,22
169,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,Las condiciones actuales no constituyen democr...,118,17
283,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",9,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,por la estructura establecida en el dialogo an...,52,8
316,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",10,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,debemos pasar de una democracia representativa...,66,9
363,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",11,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,por eso estamos aqui,20,4
491,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",14,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Se garantice una democracia vinculante que se ...,89,14
526,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",15,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,la base y el principio de todos,31,7
746,Arica y Parinacota,Parinacota,Putre,121,Liceo Granaderos de Putre,"Teniente del Campo 242, Putre",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Mejorar en todos los aspectos los sistemas de...,258,41
945,Tarapacá,Iquique,Alto Hospicio,107,San Antonio de Matilla,"Los Kiwis 3443, Alto Hospicio",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Elemento fundamental en la nueva constitución ...,54,8


### **06. Pre-Procesamiento de Texto**


#### Normalización de Texto

In [26]:
def normalize_text(text: str) -> str:
    text = str(text).lower() #
    text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8','ignore')  # eliminar tildes
    text = re.sub(r'[^a-z0-9\\s]', ' ', text) # eliminar caracteres especiales
    text = re.sub(r'\\s+', ' ', text).strip() # eliminar espacios extra
    return text

subset['texto_norm'] = subset[texto_col].apply(normalize_text)

In [32]:
subset

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw,texto_norm,doc
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2,sin fundamento,"[(sin, sin, ADP), (fundamento, fundamento, NOUN)]"
142,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",5,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Que el ciudadano pueda participar en la toma d...,141,22,que el ciudadano pueda participar en la toma d...,"[(que, que, SCONJ), (el, el, DET), (ciudadano,..."
169,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,Las condiciones actuales no constituyen democr...,118,17,las condiciones actuales no constituyen democr...,"[(las, el, DET), (condiciones, condición, NOUN..."
283,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",9,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,por la estructura establecida en el dialogo an...,52,8,por la estructura establecida en el dialogo an...,"[(por, por, ADP), (la, el, DET), (estructura, ..."
316,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",10,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,debemos pasar de una democracia representativa...,66,9,debemos pasar de una democracia representativa...,"[(debemos, deber, AUX), (pasar, pasar, VERB), ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30184,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Incorporarle nuevos elementos de acuerdo a los...,105,13,incorporarle nuevos elementos de acuerdo a los...,"[(incorporar, incorporar, VERB), (le, él, PRON..."
30212,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",3,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,La democracia debe ser la base de la sociedad ...,299,46,la democracia debe ser la base de la sociedad ...,"[(la, el, DET), (democracia, democracia, NOUN)..."
30251,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",4,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,"Descentralización, con justicia e igualdad, co...",106,15,descentralizacion con justicia e igualdad co...,"[(descentralizacion, descentralización, NOUN),..."
30290,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Se relaciona con el concepto igualitario,40,6,se relaciona con el concepto igualitario,"[(se, él, PRON), (relaciona, relacionar, VERB)..."


#### Tokenización + POS + Lematización

In [46]:
def stanza_process(doc_str: str):
    doc = nlp(doc_str)
    rows = []  # (token, lemma, upos)
    for sent in doc.sentences:
        for w in sent.words:
            rows.append((w.text, w.lemma.lower(), w.upos))
    return rows

subset['doc'] = subset['texto_norm'].progress_apply(stanza_process)

100%|██████████| 507/507 [01:18<00:00,  6.42it/s]


#### Visualización de tokens crudos (antes de filtrar stopwords) del subconjunto

In [49]:
subset['tokens_crudos'] = subset['doc'].apply(lambda toks: [lemma for _, lemma, pos in toks])

freq_crudas = Counter([w for fila in subset['tokens_crudos'] for w in fila])
top30 = freq_crudas.most_common(30)
top30

[('el', 1071),
 ('de', 600),
 ('y', 399),
 ('que', 353),
 ('ser', 328),
 ('democracia', 308),
 ('él', 278),
 ('deber', 232),
 ('uno', 225),
 ('en', 217),
 ('a', 193),
 ('participativo', 181),
 ('participación', 157),
 ('como', 116),
 ('no', 113),
 ('para', 101),
 ('con', 99),
 ('ciudadano', 92),
 ('todo', 90),
 ('representativo', 82),
 ('por', 81),
 ('valor', 77),
 ('vinculante', 72),
 ('pueblo', 65),
 ('fundamental', 60),
 ('estar', 60),
 ('poder', 56),
 ('acuerdo', 55),
 ('derecho', 53),
 ('tener', 52)]

In [38]:
subset['doc']

1        [(sin, sin, ADP), (fundamento, fundamento, NOUN)]
142      [(que, que, SCONJ), (el, el, DET), (ciudadano,...
169      [(las, el, DET), (condiciones, condición, NOUN...
283      [(por, por, ADP), (la, el, DET), (estructura, ...
316      [(debemos, deber, AUX), (pasar, pasar, VERB), ...
                               ...                        
30184    [(incorporar, incorporar, VERB), (le, él, PRON...
30212    [(la, el, DET), (democracia, democracia, NOUN)...
30251    [(descentralizacion, descentralización, NOUN),...
30290    [(se, él, PRON), (relaciona, relacionar, VERB)...
30372    [(un, uno, DET), (valor, valor, NOUN), (necesa...
Name: doc, Length: 507, dtype: object

#### Stopwords

In [39]:
spanish_sw = set(stopwords.words('spanish'))
#extra_stop = {'chile','pais','region','cabildo','provincia'}
#stopwords_total = spanish_sw.union(extra_stop)
spanish_sw

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

#### Filtrar Sustantivos

In [40]:
subset['nouns'] = subset['doc'].apply(
    lambda toks: [lemma for _, lemma, pos in toks
                  if pos in {"NOUN","PROPN"} and lemma not in spanish_sw and len(lemma) > 2]
)

In [37]:
subset['nouns']

1                                             [fundamento]
142      [ciudadano, toma, decisión, igualdad, aceptaci...
169      [condición, democracia, obstante, valor, const...
283                                  [estructura, dialogo]
316                                           [democracia]
                               ...                        
30184                          [elemento, acuerdo, tiempo]
30212    [democracia, base, sociedad, modalidad, gobier...
30251    [descentralización, justicia, igualdad, opinió...
30290                                           [concepto]
30372    [valor, sistema, ciudadanía, aumento, priorida...
Name: nouns, Length: 507, dtype: object

#### Filtrar Adjetivos

In [42]:

subset['adjs'] = subset['doc'].apply(
    lambda toks: [lemma for _, lemma, pos in toks
                  if pos == 'ADJ' and len(lemma) > 2]
)

In [45]:
subset

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw,texto_norm,doc,nouns,adjs
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2,sin fundamento,"[(sin, sin, ADP), (fundamento, fundamento, NOUN)]",[fundamento],[]
142,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",5,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Que el ciudadano pueda participar en la toma d...,141,22,que el ciudadano pueda participar en la toma d...,"[(que, que, SCONJ), (el, el, DET), (ciudadano,...","[ciudadano, toma, decisión, igualdad, aceptaci...",[]
169,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,Las condiciones actuales no constituyen democr...,118,17,las condiciones actuales no constituyen democr...,"[(las, el, DET), (condiciones, condición, NOUN...","[condición, democracia, obstante, valor, const...","[actual, fundamental, nuevo]"
283,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",9,¿Cuáles son los valores y principios más impor...,Democracia,NaN,P,por la estructura establecida en el dialogo an...,52,8,por la estructura establecida en el dialogo an...,"[(por, por, ADP), (la, el, DET), (estructura, ...","[estructura, dialogo]","[establecido, anterior]"
316,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",10,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,debemos pasar de una democracia representativa...,66,9,debemos pasar de una democracia representativa...,"[(debemos, deber, AUX), (pasar, pasar, VERB), ...",[democracia],"[representativo, participativo]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30184,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",2,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Incorporarle nuevos elementos de acuerdo a los...,105,13,incorporarle nuevos elementos de acuerdo a los...,"[(incorporar, incorporar, VERB), (le, él, PRON...","[elemento, acuerdo, tiempo]","[nuevo, ademocracia, sustentable, sostenible, ..."
30212,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",3,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,La democracia debe ser la base de la sociedad ...,299,46,la democracia debe ser la base de la sociedad ...,"[(la, el, DET), (democracia, democracia, NOUN)...","[democracia, base, sociedad, modalidad, gobier...","[chileno, democratico, fundamental, directo, r..."
30251,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",4,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,"Descentralización, con justicia e igualdad, co...",106,15,descentralizacion con justicia e igualdad co...,"[(descentralizacion, descentralización, NOUN),...","[descentralización, justicia, igualdad, opinió...",[]
30290,Magallanes y de la Antártica Chilena,Última Esperanza,Natales,144,Liceo Bernardo O'Higgins,"Eleuterio Ramirez 748, Natales",6,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Se relaciona con el concepto igualitario,40,6,se relaciona con el concepto igualitario,"[(se, él, PRON), (relaciona, relacionar, VERB)...",[concepto],[igualitario]
